In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import copy
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from method.models import generate_random_values, generate_static_values, generate_llm_values
from method.ours.utils import create_driver, get_xpath, interact_with_input
from method.ours.history import HistoryTable
from method.ours.feedback import get_global_feedback

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

HEADLESS = False
URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
METHOD = 'llm'

In [5]:
def get_to_form(driver):
    driver.get(URL)
    
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()

In [6]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [7]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = driver.find_elements(By.TAG_NAME, 'form')[1]
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [8]:
history_table = HistoryTable(
    url=URL,
    xpath=form_xpath
)

In [9]:
values = None
if METHOD == 'static':
    values = generate_static_values(driver, inputs)
elif METHOD == 'random':
    values = generate_random_values(driver, inputs)
else:
    values = generate_llm_values(form.get_attribute('outerHTML'), openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate testing values for a web form. When given a web form, you need to generate values that can pass and fail when tried on the form. Your response should be in the following format:\n{\n[field_identifier]: {\npassing: <a base passing value for the field>,\nfailing: <a list of failing values for the field>\n}\n}\nYour response must be parsable by Python's json.loads function. Use \"id\" attribute to refer to the fields.\n"
  },
  {
    "role": "user",
    "content": "<form novalidate=\"\" autocomplete=\"off\" class=\"bkmg-tab-content-container bkmg-flights-tab-container ng-dirty ng-invalid ng-touched\"><fieldset><legend class=\"visually-hidden\"> Flights </legend><div class=\"bkmg-flights-tab-grid bkmg-flights-tab-grid-multi-city\"><fieldset class=\"bkmg-flights-tab-trip-type-selector mobile-padding\"><legend class=\"visually-hidden\"> Trip type </legend><abc-radio-group formcontrolname=\"tripType\" class=\"ng-

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 9178 tokens. Please reduce the length of the messages.

In [13]:
values

{'//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[1]/DIV[1]/INPUT[1]': {'passing': True,
  'failing': []},
 '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[2]/DIV[1]/INPUT[1]': {'passing': True,
  'failing': []},
 '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[3]/DIV[1]/INPUT[1]': {'passing': True,
  'failing': []},
 '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MA

In [15]:
def send_values(elements, values):
    for element_id in elements.keys():
        element = elements[element_id]
        value = values[element_id]
        
        element_type = element.get_attribute('type') or 'text'
    
        if element_type in ['radio', 'checkbox'] or element.tag_name in ['button']:
            continue
        
        interact_with_input(element, value)

        
def submit_form(form):
    submit = form.find_element(By.XPATH, f'{form_xpath}//*[@type="submit"]')
    
    for _ in range(3):
        try:
            interact_with_input(submit, True)
            time.sleep(0.5)
        except:
            break

In [16]:
passing_set = {
    element_id: values[element_id]['passing'] for element_id in values.keys()
}


for element_id, element_values in values.items():
    get_to_form(driver)
    
    elements = {
        element_id: driver.find_element(
            By.ID if METHOD == 'llm' else By.XPATH, element_id
        ) for element_id in values.keys()
    }
    element = elements[element_id]
    element_type = element.get_attribute('type') or 'text'
    
    if element_type in ['radio', 'checkbox']:
        continue
    
    for failing_value in element_values['failing']:
        try:
            passing_copy = copy.copy(passing_set)
            passing_copy[element_id] = failing_value
            send_values(elements, passing_copy)

            time.sleep(2)

            submit_form(form)

            new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
            global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

            history_table.add(
                passing_copy,
                'base',
                global_feedback,
                driver.current_url
            )
        except Exception as e:
            print(e)

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

...

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000109e9f6b8 chromedriver + 4937400
1   chromedriver                        0x0000000109e96b73 chromedriver + 4901747
2   chromedriver                        0x0000000109a54616 chromedriver + 435734
3   chromedriver                        0x0000000109a6466d chromedriver + 501357
4   chromedriver                        0x0000000109a5a1b3 chromedriver + 459187
5   chromedriver                        0x0000000109a586e1 chromedriver + 452321
6   chromedriver                        0x0000000109a5bb87 chromedriver + 465799
7   chromedriver                        0x0000000109a5bc8c chromedriver + 466060
8   chromedriver                        0x0000000109a97a2b chromedriver + 711211
9   chromedriver                        0x0000000109a980a1 chromedriver + 712865
10  chromedriver                        0x0000000109a8bae6 chromedriver + 6622

KeyboardInterrupt: 

In [16]:
for feedback in history_table.table.feedback.unique():
    print(feedback)
    print()

Skip to book a flight  Skip to main navigation  Skip to content  Skip to search field  Skip to footer links  Air Canada Search Search 



AccessibilitySelect your edition and language. You are currently on the Canada English edition.   You will be billed in  Canadian dollars  - CA$EnglishCA$







 Sign in 



Book



Plan



Fly



Aeroplan



Customer supportSpecial offers Air Canada - Official website - Homepage  Top of page banner 2X THE POINTSKonnichiwa, JapanSay hi to 2X the Aeroplan points on flights between Canada and Japan. Book by July 26 for travel until December 15, 2023.Register and bookSALE ON NOWSave on flights within CanadaUntil July 16, take advantage of savings for summer or fall travel. Book nowFlightsPackagesFlight PassesHotelsCarsMy bookingsCheck-inFlight statusAC VacationsBook your AC Vacation package  Flights  Trip type  Round-trip  One-way  Multi-city/Stopover  Book with points Vancouver YVRFrom Enter the first three characters of the city, airport, province, s